In [2]:
import tensorflow as tf
import os
from openvino.inference_engine import IENetwork, IEPlugin, IECore
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [3]:
# path to Dataset Dir
workspace_dir = '/home/manuel/Bachelor_Arbeit/workspace'
labels = []

# 1. Dataset and Model
dataset = 'OI_Animals_Augmented_9_2000/'
model = 'faster_rcnn_inception_v2_coco_2018_01_28_out/'
model_xml = os.path.join(workspace_dir, dataset, model, 'open_vino', 'frozen_inference_graph.xml')
model_bin = os.path.join(workspace_dir, dataset, model, 'open_vino', 'frozen_inference_graph.bin')
labels.append([l.strip() for l in open(os.path.join(workspace_dir, dataset, 'classes.txt')).readlines()])

# 2. Dataset and Model
dataset2 = 'Beispiel_Set'
model_2 = 'ssd_inception_v2_coco_out'
model_2_xml = os.path.join(workspace_dir, dataset2, model_2, 'open_vino', 'frozen_inference_graph.xml')
model_2_bin = os.path.join(workspace_dir, dataset2, model_2, 'open_vino', 'frozen_inference_graph.bin')
labels.append([l.strip() for l in open(os.path.join(workspace_dir, dataset2, 'classes.txt')).readlines()])

# path to images to infer
test_images_dir = '/home/manuel/Bachelor_Arbeit/test'

In [20]:
# get all image paths
if os.path.isdir(test_images_dir):
    images = [
        os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img[-3:] == 'jpg']
    print(str(len(images)) + ' images found')
    #shuffle(images)
elif os.path.isfile(test_images_dir) and test_images_dir[-3:] == 'jpg':
    print('one image found')
    images = [test_images_dir]
else:
    print('wrong input')
    exit()

14 images found


In [5]:
# create one plugin
ie = IECore()

# create two networks
net = [IENetwork(model=model_xml, weights=model_bin), IENetwork(model=model_2_xml, weights=model_2_bin)]
model_names = ['fasterrcnn animals', 'ssd sample']

In [6]:
# load the two executable nets from the plugin
exec_net = [ie.load_network(network=net[0],
                            num_requests=2,
                            device_name='MYRIAD'),
            ie.load_network(network=net[1],
                            num_requests=2,
                            device_name='MYRIAD')]


In [21]:
# initialize pairs needed varables 0:first_model, 1:second_model
img_info_input_blob = [None, None]
feed_dict = [{}, {}]
input_blob = [None, None]
img_info_input_blob = [None, None]
out_blob = [None, None]
nchw = [None, None]
cur_request_id = [0, 0]
next_request_id = [1, 1]
frame = [cv2.imread(images[0]), cv2.imread(images[0])]
initial_h = [frame[0].shape[0], frame[1].shape[0]]
initial_w = [frame[0].shape[1], frame[1].shape[1]]
next_frame = [None, None]
next_initial_h = [None, None]
next_initial_w = [None, None]
detection_results = [None, None]

# create for both networks the in- outputblobs
for i in range(2):
    for blob_name in net[i].inputs:
        if len(net[i].inputs[blob_name].shape) == 4:
            input_blob[i] = blob_name
        elif len(net[i].inputs[blob_name].shape) == 2:
            img_info_input_blob[i] = blob_name
        else:
            raise RuntimeError("Unsupported {}D input layer '{}'. Only 2D and 4D input layers are supported"
                                   .format(len(net.inputs[blob_name].shape), blob_name))

    assert len(net[i].outputs) == 1, "Demo supports only single output topologies"

    out_blob[i] = next(iter(net[i].outputs))
    
    nchw[i] = net[i].inputs[input_blob[i]].shape
    
    if img_info_input_blob[i]:
        feed_dict[i][img_info_input_blob[i]] = [nchw[i][2], nchw[i][3], 1]


In [26]:
# create a frame with h,w for the first run (will not be infered)
frame = [cv2.imread(images[0]), cv2.imread(images[0])]
initial_h = [frame[0].shape[0], frame[1].shape[0]]
initial_w = [frame[0].shape[1], frame[1].shape[1]]
for img_path in images:
    
    # set index for the first Model
    i = 0
    
    # read the image from path
    next_frame[i] = cv2.imread(img_path)
    next_initial_h[i], next_initial_w[i] = next_frame[i].shape[: 2]
    
    # resize the image
    in_frame = cv2.resize(next_frame[i], (nchw[i][3], nchw[i][2]))
    in_frame = in_frame.transpose((2, 0, 1))
    in_frame = in_frame.reshape((nchw[i][0], nchw[i][1], nchw[i][2], nchw[i][3]))
    
    # start async request for next frame
    feed_dict[i][input_blob[i]] = in_frame
    exec_net[i].start_async(request_id=next_request_id[i], inputs=feed_dict[i])
    
    # check for current frame
    if exec_net[i].requests[cur_request_id[i]].wait(-1) == 0:

        # get infer results from current frame
        res = exec_net[i].requests[cur_request_id[i]].outputs[out_blob[i]]
        detection_results[i] = []
        for obj in res[0][0]:
            if obj[2] > 0.5:
                xmin = int(obj[3] * initial_w[i])
                ymin = int(obj[4] * initial_h[i])
                xmax = int(obj[5] * initial_w[i])
                ymax = int(obj[6] * initial_h[i])
                class_id = int(obj[1])
                color = (min(class_id * 12.5, 255),
                             min(class_id * 7, 255), min(class_id * 5, 255))
                cv2.rectangle(frame[i], (xmin, ymin),
                                  (xmax, ymax), (255, 255, 0), 2)
                det_label = labels[i][class_id - 1]
                detection_results[i].append(det_label)
                cv2.putText(frame[i], det_label + ' ' + str(round(obj[2] * 100, 1)) + ' %', (xmin, ymin - 7),
                                cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 0), 1)
    
    # print/show result of infered current frame 
    cv2.imshow("Detection Results", frame[i])
    print(model_names[i])
    print(detection_results[i])
    
    # save a copy of the infered current frame for the next Model as Input
    frame_tmp = np.copy(frame[i])
    
    # swithch request id's
    cur_request_id[i], next_request_id[i] = next_request_id[i], cur_request_id[i]
    
    # current <- next
    frame[i] = next_frame[i]
    initial_w[i] = next_initial_w[i]
    initial_h[i] = next_initial_h[i]
    
    key = cv2.waitKey(0)
    
    
    
    
    # set index for the sectond Model
    # Model 2 ist immer um 2 Bilder hinterher
    i = 1
    
    # take the Output from Model_1 as Input
    next_frame[i] = frame_tmp
    next_initial_h[i], next_initial_w[i] = next_frame[i].shape[: 2]
    
    # resize the image
    in_frame = cv2.resize(next_frame[i], (nchw[i][3], nchw[i][2]))
    in_frame = in_frame.transpose((2, 0, 1))
    in_frame = in_frame.reshape((nchw[i][0], nchw[i][1], nchw[i][2], nchw[i][3]))
    
    # start async request for next frame
    feed_dict[i][input_blob[i]] = in_frame
    exec_net[i].start_async(request_id=next_request_id[i], inputs=feed_dict[i])
    
    # check for current frame
    if exec_net[i].requests[cur_request_id[i]].wait(-1) == 0:

        # get infer results from current frame
        res = exec_net[i].requests[cur_request_id[i]].outputs[out_blob[i]]
        detection_results[i] = []
        for obj in res[0][0]:
            if obj[2] > 0.5:
                xmin = int(obj[3] * initial_w[i])
                ymin = int(obj[4] * initial_h[i])
                xmax = int(obj[5] * initial_w[i])
                ymax = int(obj[6] * initial_h[i])
                class_id = int(obj[1])
                color = (min(class_id * 12.5, 255),
                             min(class_id * 7, 255), min(class_id * 5, 255))
                cv2.rectangle(frame[i], (xmin, ymin),
                                  (xmax, ymax), (255, 255, 0), 2)
                det_label = labels[i][class_id - 1]
                detection_results[i].append(det_label)
                cv2.putText(frame[i], det_label + ' ' + str(round(obj[2] * 100, 1)) + ' %', (xmin, ymin - 7),
                                cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 0), 1)
    
    # print/show result of infered current frame 
    cv2.imshow("Detection Results", frame[i])
    print(model_names[i])
    print(detection_results[i])
    
    # save a copy of the infered current frame for the next Model as Input
    frame_tmp = np.copy(frame[i])
    
    # swithch request id's
    cur_request_id[i], next_request_id[i] = next_request_id[i], cur_request_id[i]
    
    # current <- next
    frame[i] = next_frame[i]
    initial_w[i] = next_initial_w[i]
    initial_h[i] = next_initial_h[i]
    
    key = cv2.waitKey(0)

    if key == 27:
        break

cv2.destroyAllWindows()

fasterrcnn animals
[]
ssd sample
[]
fasterrcnn animals
[]
ssd sample
['Human_head']
fasterrcnn animals
['Deer', 'Deer', 'Deer']
ssd sample
['Human_head']
fasterrcnn animals
['Deer', 'Deer', 'Deer', 'Deer', 'Deer']
ssd sample
[]
fasterrcnn animals
['Deer', 'Deer', 'Deer', 'Deer', 'Deer']
ssd sample
[]
fasterrcnn animals
[]
ssd sample
[]
fasterrcnn animals
['Deer', 'Deer', 'Deer']
ssd sample
['Human_head']
fasterrcnn animals
[]
ssd sample
['Mobile_phone']
fasterrcnn animals
['Deer', 'Deer']
ssd sample
['Human_head']
fasterrcnn animals
[]
ssd sample
[]
fasterrcnn animals
['Deer']
ssd sample
['Human_head']
